# Trade Performance Data Processor

*This script pulls in Ninja Trader Performance Data CSVs, sorts them by trade open date & trade close date, and then writes the data to a new CSV called training_data_{date_range_start}_{date_range_end}.csv*

In [1]:
import csv
from datetime import datetime
import re
from statistics import fmean
import activators
import warnings

date_format = '%m/%d/%Y %H:%M:%S'

In [2]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
trades = []

In [4]:
def unpack_trades(file_name):
    trades = []
    with open(file_name, newline='') as f:
        reader = csv.DictReader(f)

        for row in reader:
            # print(row)
            trades.append(row)

    return trades

def parse_date(date_str):
    return datetime.strptime(date_str, date_format)

In [5]:
live_trades = unpack_trades("Ninja_Trader_Performance_Jabari_Live_10-6-2023_11-17-2023.csv")

len(live_trades)

58

In [6]:
sim_trades = unpack_trades("Ninja_Trader_Performance_Jabari_Simulator_10-6-2023_11-17-2023.csv")

len(sim_trades)

79

In [7]:
trades += live_trades
trades += sim_trades

len(trades)
trades[1]

{'symbol': 'MCLZ3',
 '_priceFormat': '-2',
 '_priceFormatType': '0',
 '_tickSize': '0.01',
 'buyFillId': '3429478032',
 'sellFillId': '3429478041',
 'qty': '1',
 'buyPrice': '88.87',
 'sellPrice': '89.11',
 'pnl': '$24.00',
 'boughtTimestamp': '10/20/2023 07:46:29',
 'soldTimestamp': '10/20/2023 07:48:34',
 'duration': '2min 5sec'}

In [8]:
trades.sort(key=lambda x: parse_date(x['boughtTimestamp']) if parse_date(x['boughtTimestamp']) < parse_date(x['soldTimestamp']) else parse_date(x['soldTimestamp']))

len(trades)

137

In [9]:
#format trade pnl
for trade in trades:
    formatted_pnl = re.sub(r'[$),]', '',trade["pnl"])
    formatted_pnl = re.sub(r'[(]', '-',formatted_pnl)
    trade['pnl'] = formatted_pnl
    trade['open_datetime'] = trade_open_datetime = parse_date(trade['boughtTimestamp']) if parse_date(trade['boughtTimestamp']) < parse_date(trade['soldTimestamp']) else parse_date(trade['soldTimestamp'])
    trade['close_datetime'] = parse_date(trade['boughtTimestamp']) if parse_date(trade['boughtTimestamp']) > parse_date(trade['soldTimestamp']) else parse_date(trade['soldTimestamp'])

trades[0]

{'symbol': 'MESZ3',
 '_priceFormat': '-2',
 '_priceFormatType': '0',
 '_tickSize': '0.25',
 'buyFillId': '23603390017',
 'sellFillId': '23603390007',
 'qty': '1',
 'buyPrice': '4393.50',
 'sellPrice': '4393.50',
 'pnl': '0.00',
 'boughtTimestamp': '10/10/2023 21:09:16',
 'soldTimestamp': '10/10/2023 20:59:23',
 'duration': '9min 53sec',
 'open_datetime': datetime.datetime(2023, 10, 10, 20, 59, 23),
 'close_datetime': datetime.datetime(2023, 10, 10, 21, 9, 16)}

In [10]:
# stats
wins = []

for trade in trades:
    pnl = float(trade['pnl'])
    if pnl >= 0:
        wins.append(pnl)

avg_profit = fmean(wins)

avg_profit

# scale/normalize to pnl_per_contract / avg_profit and add as reward/punishment to the NN inputs
# to encourage the trade planner ML model towards more profitable trades and away from more
# loss likely trades (e.g. losing $780 on NQ, will subtract ~15points from my trade plan score
# as punishment, which feels like a lot, but earning $400 on NQ will add back 8 points
# so it's pretty normalized(?). not sure yet. maybe scale to points(?). Remember that pnl are effects
# not causes so the amount of profit is not as important as the repeatability and
# probability of profit based on causes

50.02564102564103

In [11]:
range_start_date = parse_date(trades[0]['boughtTimestamp']) if parse_date(trades[0]['boughtTimestamp']) < parse_date(trades[0]['soldTimestamp']) else parse_date(trades[0]['soldTimestamp'])

range_start_date = range_start_date.strftime("%Y-%m-%d")

range_end_date = parse_date(trades[len(trades) - 1]['boughtTimestamp']) if parse_date(trades[len(trades) - 1]['boughtTimestamp']) < parse_date(trades[len(trades) - 1]['soldTimestamp']) else parse_date(trades[len(trades) - 1]['soldTimestamp'])

range_end_date = range_end_date.strftime("%Y-%m-%d")

In [12]:
instrument_qualities = activators.calc_instrument_qualities(trades)

In [13]:
trades = activators.calc_instrument_condition_qualities(trades)
trades.sort(key=lambda x: parse_date(x['boughtTimestamp']) if parse_date(x['boughtTimestamp']) < parse_date(x['soldTimestamp']) else parse_date(x['soldTimestamp']))
#print(trades)

In [14]:
trades = activators.calc_market_condition_qualities(trades)
trades.sort(key=lambda x: parse_date(x['boughtTimestamp']) if parse_date(x['boughtTimestamp']) < parse_date(x['soldTimestamp']) else parse_date(x['soldTimestamp']))
#print(trades)

In [15]:
trades = activators.calc_time_of_day_qualities(trades)
trades.sort(key=lambda x: parse_date(x['boughtTimestamp']) if parse_date(x['boughtTimestamp']) < parse_date(x['soldTimestamp']) else parse_date(x['soldTimestamp']))
#print(trades)

['9-10']
['8-9', '12-13']
['8-9']
['9-10', '8-9']
['9-10']
['8-9']


In [16]:
trades = activators.calc_position_size_qualities(trades)
trades.sort(key=lambda x: parse_date(x['boughtTimestamp']) if parse_date(x['boughtTimestamp']) < parse_date(x['soldTimestamp']) else parse_date(x['soldTimestamp']))
#print(trades)

1.15
1.0
1.1
2.0
1.0
3.5


In [17]:
trades = activators.calc_profitability_qualities(trades)

In [19]:
file_name = f"training_data_{range_start_date}_{range_end_date}.txt"

with open(file_name, 'w', newline='') as csvfile:
    # add "Is_Would_Have_Been_Profitable" to V1 after trade plan structure is in place
    fieldnames = ["Trade_Id","Symbol","Trade_Open_Datetime","Trade_Close_Datetime","Trade_Quantity","Trade_Direction","Trade_Open_Price","Trade_Close_Price","Trade_PnL", "Is_Good_Mood","Is_Good_Energy","Is_Revenge_Trade","Has_Trade_Plan", "Is_Automated_Trade_Mgmt", "Is_Alert_Based", "Is_Good_Trade_Setup","Has_Confirmations","Has_Good_Confirmations","Has_Good_Risk_Reward_Ratio","Is_Profitable_Trade","Has_Followed_Plan_Completely","Has_Followed_Plan_Partially","Has_Exited_Early","Has_Stopped_Out_Late","Has_Not_Followed_Plan_At_All","Is_Good_Time_Of_Day_To_Trade","Is_Good_Market_Conditions","Is_Good_Instrument_Conditions","Is_Good_Financial_Instrument","Is_Okay_Financial_Instrument","Is_Bad_Financial_Instrument","Is_Good_Position_Size","Label_Is_High_Prob_Trade","Label_Is_Med_Prob_Trade","Label_Is_Low_Prob_Trade"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    
    for i, trade in enumerate(trades):
        trade_open_datetime = trade['open_datetime'].isoformat()
        trade_close_datetime = trade['close_datetime'].isoformat()

        trade_direction = "Long"
        open_price = trade["buyPrice"]
        close_price = trade["sellPrice"]

        if float(trade['pnl']) < 0:
            if float(trade["buyPrice"]) > float(trade["sellPrice"]):
                trade_direction = "Short"
                open_price = trade["sellPrice"]
                close_price = trade["buyPrice"]
        else:
            if float(trade["buyPrice"]) < float(trade["sellPrice"]):
                trade_direction = "Short"
                open_price = trade["sellPrice"]
                close_price = trade["buyPrice"]

        market_conditions_quality = trade['market_conditions_quality']
        instrument_quality = instrument_qualities[trade['symbol']]
        instrument_conditions_quality = trade['instrument_conditions_quality']
        time_of_day_quality = trade['time_of_day_quality']
        position_size_quality = trade['position_size_quality']
        profitability_quality = trade['profitability_quality']

        is_good_financial_instrument = 1 if instrument_quality == activators.QualityType.HIGH else 0
        is_okay_financial_instrument = 1 if instrument_quality == activators.QualityType.MED else 0
        is_bad_financial_instrument = 1 if instrument_quality == activators.QualityType.LOW else 0

        is_good_market_conditions = 1 if market_conditions_quality == activators.QualityType.HIGH else 0
        is_good_instrument_conditions = 1 if instrument_conditions_quality == activators.QualityType.HIGH else 0
        is_good_time_of_day_to_trade = 1 if time_of_day_quality == activators.QualityType.HIGH else 0
        is_good_position_size_quality = 1 if position_size_quality == activators.QualityType.HIGH else 0
        is_profitable_trade = 2 if profitability_quality == activators.QualityType.HIGH else -1
        

        datarow = {
                    "Trade_Id": i + 1, 
                    "Symbol": trade['symbol'], 
                    "Trade_Open_Datetime": trade_open_datetime, 
                    "Trade_Close_Datetime": trade_close_datetime, 
                    "Trade_Quantity": trade['qty'], 
                    "Trade_Direction": trade_direction, 
                    "Trade_Open_Price" : open_price, 
                    "Trade_Close_Price": close_price, 
                    "Trade_PnL": trade['pnl'],
                    "Is_Good_Financial_Instrument": is_good_financial_instrument,
                    "Is_Okay_Financial_Instrument": is_okay_financial_instrument,
                    "Is_Bad_Financial_Instrument": is_bad_financial_instrument,
                    "Is_Good_Instrument_Conditions": is_good_instrument_conditions,
                    "Is_Good_Time_Of_Day_To_Trade": is_good_time_of_day_to_trade,
                    "Is_Good_Market_Conditions": is_good_market_conditions,
                    "Is_Good_Position_Size": is_good_position_size_quality,
                    "Is_Profitable_Trade": is_profitable_trade,
                    "Is_Good_Trade_Setup": 0, # missing data
                    "Has_Confirmations": 0, # missing data
                    "Has_Good_Confirmations": 0, # missing data
                    "Has_Good_Risk_Reward_Ratio": 0, # missing data
                    "Is_Good_Mood": 1, # mark as 1 as default to make data editing easy
                    "Is_Good_Energy": 1,  # mark as 1 as default to make data editing easy
                    "Is_Revenge_Trade": -1, # mark as 0 as default to make data editing easy
                    "Is_Automated_Trade_Mgmt": 0, # mark as 0 as default to make data editing easy
                    "Is_Alert_Based": 0, # mark as 0 as default to make data editing easy
                    "Has_Trade_Plan": 0, # mark as 0 as default to make data editing easy
                    "Has_Followed_Plan_Completely": 0, # mark as 0 as default to make data editing easy
                    "Has_Followed_Plan_Partially": 0, # mark as 0 as default to make data editing easy
                    "Has_Exited_Early": 0, # mark as 0 as default to make data editing easy
                    "Has_Stopped_Out_Late": 0, # mark as 0 as default to make data editing easy
                    "Has_Not_Followed_Plan_At_All": 0, # mark as 0 as default to make data editing easy
                    }
            
        writer.writerow(datarow)
        